# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)

[Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)


In [ ]:
!pip install -q -U langchain transformers bitsandbytes accelerate

In [ ]:
import numpy as np
import json
from datasets import load_dataset
from random import shuffle
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import gc

In [ ]:
good_responses, letters = None, None
def set_letters(k):
    global letters, good_responses
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'][:k]
    good_responses = [f'{x})' for x in letters]
set_letters(4)
print(letters)
print(good_responses)

## Metrike

Preuzeto direktno s njihovog repoa

In [ ]:
import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

def getResultdata(result_data):
    global letters
    choice_to_index = {letters[i]: i for i in range(len(letters))}
    choice_to_index[None] = len(letters)

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1
                
    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)
    
    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    word_data_list = list(word_play.values())
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)
    
    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')   
    final_result['sentence'] = getMetric(reverse_data_list)  
    
    
    all_data = word_data_list + reverse_data_list
    print('#########All data##########') 
    final_result['all'] = getMetric(all_data) 
    
    return final_result

### Spremanje datoteka

Automatski dodaje +1 na naziv ako vec postoji takva datoteka

Uzima format model_name + name + redni broj za datoteku

Unutra pise prvi testni primjer (example) i rezultate

In [ ]:
dirname = 'resultsnew'
pickle_dir = 'pickled_results'

def create_dir(name):
    """Create a directory if it doesn't exist."""
    try:
        os.mkdir(name)
    except FileExistsError:
        pass

def tensor_to_serializable(obj):
    """
    Helper function to convert non-serializable objects like PyTorch tensors 
    into a JSON serializable format.
    """
    if isinstance(obj, torch.Tensor):
        return obj.tolist()  # Convert tensors to lists
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

def save(final_result, name=None, few_shot=False, example=None, data=None):
    """Save the final result and data into text and JSON files."""
    create_dir(dirname)
    create_dir(pickle_dir)
    
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    
    while True:
        file_path = f'{dirname}/{model_name}{inserted_name}_{i}.txt'
        pickle_path = f'{pickle_dir}/{model_name}{inserted_name}_{i}.json'
        try:
            with open(file_path, 'r'):
                i += 1
        except FileNotFoundError:
            # Writing the results to a text file
            with open(file_path, 'w') as file:
                file.write(model_name + '\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if example:
                    if not isinstance(example, str):
                        example = json.dumps(example, indent=4)
                    file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4, default=tensor_to_serializable))
            
            # Writing data to a JSON file
            if data is None:
                print(f"Warning: {model_name} results data not saving in json, please provide data arg")
            else:
                with open(pickle_path, 'w') as file:
                    file.write(json.dumps(data, indent=4, default=tensor_to_serializable))
            
            return


### Load the model

Login kako bi se mogli ucitati napredniji modeli

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

Logiranje u huggingiface
treba kljuc, njega nabavite na huggingface stranici

In [ ]:
from huggingface_hub import login
login('', add_to_git_credential=False, write_permission=True)

BitsAndBytesConfig je novi, to kao kvantizira model pa stane na graficku

In [ ]:
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
# model_path = 'mistralai/Mistral-7B-v0.3'
# model_path = 'microsoft/phi-2'
# model_path = 'microsoft/Phi-3-mini-128k-instruct'
# model_path = 'microsoft/Phi-3-mini-4k-instruct'
# model_path = '01-ai/Yi-1.5-6B'
# model_path = 'netcat420/MFANN3bv0.6'
model_path = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_path = "meta-llama/Meta-Llama-3-8B"
# model_path = "allenai/OLMo-7B-Instruct"
# model_path = "tiiuae/falcon-7b-instruct"
# model_path = "tiiuae/falcon-7b"
# model_path = 'Intel/neural-chat-7b-v3-1'
# model_path = "google/gemma-1.1-7b-it"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def get_mistral():
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True,
    )
    return tokenizer, model

def get_llama():
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model_4bit = AutoModelForCausalLM.from_pretrained(
        model_path,
#         "SweatyCrayfish/llama-3-8b-quantized", 
#         load_in_4bit=True, 
#         torch_dtype=torch.float16,
        device_map="auto",
        quantization_config=quantization_config,
    )
    return tokenizer, model_4bit

tokenizer, model = get_llama()

### Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`, ako na Kaggleu, onda napraviti svoj dataset te ga dodati pod input

In [ ]:
# few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
data_path = '/kaggle/input/brainteaser/data'
sentence_data_path = f"{data_path}/SP-train.npy"
wordplay_data_list = f"{data_path}/WP-train.npy"
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

test_data_list = sentence_data_list + wordplay_data_list
print(f"Dataset length {len(test_data_list)}")

Format za mystral, za ostale modele maknuti linije `<s>[INST]`

In [ ]:
format = ""
def get_format():
    global format, letters
    newline = '\n'
    format = f"""Question: {"{}"}

Choices:
{''.join('(' + x + ') {}' + newline for x in letters)}"""
    return format

def get_llama_format(demonstration=None):
    global tokenizer, format
    messages = []
    if demonstration is not None:
        messages.append({"role": "system", "content": demonstration})
    messages.append({"role": "user", "content": get_format()})
    messages.append({"role": "assistant", "content": "Answer:("})
    try:
        format = tokenizer.apply_chat_template(messages, tokenize=False)
    except Exception:
        # When using mistral model, roles in messages must alternate, this way it works without errors
        messages[1]["content"] = demonstration + "\n\n" + messages[1]["content"]
        format = tokenizer.apply_chat_template(messages[1:], tokenize=False)
    # remove "<|eot_id|>" from the end
    format = format[:format.index("Answer:(")+8]

def get_appended_format(demonstration=None):
    global format
    format = get_format()
    if demonstration is not None:
        format = demonstration + "\n\n" + format
    format += "\nAnswer:("

# get_llama_format("Primjer")
set_letters(4)
get_appended_format()
print(format)

In [ ]:
def get_single_demo(sample):
    global format
    sample_demo = format.format(
        sample['question'], *sample['choice_list'])
    return sample_demo

## Generiranje dataseta

### Pomocne funkcije

In [ ]:
def set_predictions(data_list):
    """Uzima response i parsira ga tako da pod predict stavi slovo koje je napisano"""
    global letters
    for index,item in enumerate(data_list):
        item['predict'] = None
        for x in letters:
            if (f'{x})') in item['response']:
                item['predict'] = x

        if item['predict'] is None:
            print(index)

def custom_stopping_criteria(input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs) -> bool:
    """
    Funkcija kako bi se dinamicki prepoznalo kad treba prestati generirati tekst:
    ne nakon fiksnog broja tokena, vec kad model napise rjesenje. Najcesce
    ce to ipak biti medu prva dva tokena
    """
    global good_responses
    decoded = tokenizer.decode(input_ids[0][-3:])
    for good_response in good_responses:
        if good_response in decoded:
            return True
    return False

stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [ ]:
def generate(samples, tokens=10, all_tokens=True, few_shot=False):
    global model
    for sample in tqdm(samples):
        # Prepare the input text
        if few_shot:
            text = demonstration + get_single_demo(sample)
        else:
            text = get_single_demo(sample)
        
        # Encode the input using the tokenizer
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(device)
        
        # Generate the output text
        with torch.no_grad():
            results = model(inputs)
            # Get the logits and calculate the softmax probabilities for the last token
            logits = results.logits.squeeze()[-1]
            probs = torch.nn.functional.softmax(logits, dim=-1)  # Apply softmax to get probabilities
            sample['probs'] = probs  # Store probabilities in the sample

        # Generate the final response (text)
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            max_new_tokens=tokens,
            stopping_criteria=stopping_criteria
        )
        
        # Extract the generated output
        original_tokens = len(inputs[0])
        generated_output = outputs[0][0 if all_tokens else original_tokens:]
        response = tokenizer.decode(generated_output, skip_special_tokens=True)
        
        # Store the response in the sample
        sample['response'] = response
    
    # Process the predictions (e.g., based on the model outputs)
    set_predictions(samples)


In [ ]:
print(test_data_list[0])

In [ ]:
def turn_to_gibberish():
    output_name = "gibberish_all"
    
    random.seed(10)
    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i != dict["label"]:
                s = choice_list[i]
                choice_list[i] = ''.join(random.sample(s,len(s)))
        
        if dict["id"] == "SP-88":
            for key,item in dict.items():
                print(f"{key}: {item}")
    
    return test_data_list, output_name
generated_data_list, output_name = turn_to_gibberish()

In [ ]:
generate(test_data_list, tokens=20, all_tokens=False, few_shot=False)

In [ ]:
def get_probs(data, dict_matrix, threshold):
    letters = ['a', 'A', 'b', 'B', 'c', 'C', 'd', 'D']
    total_probs = {}  # Dictionary to store the cumulative probabilities for each letter
    count = 0
    total_prob = 0
    
    probs = data['probs']  # Assuming this contains probabilities from the model
    cond = probs > probs.max() * 0.0001  # Filter out very low probabilities
    indices = cond.nonzero()  # Get indices of significant probabilities
    values = probs[cond]  # Get the values of those probabilities
    
    # Sum the probabilities for each relevant letter
    for ind, val in zip(indices[:, 0], values):
        letter = tokenizer.decode(ind)
        if letter not in total_probs:
            total_probs[letter] = 0  # Initialize the letter if not already present
        total_probs[letter] += float(val)  # Add the probability for the letter
        total_prob += val  # Sum up the total probabilities
    
    # Normalize total_probs by dividing each by total_prob
    if float(total_prob) > 0:  # Prevent division by zero
        for letter in total_probs:
            total_probs[letter] /= total_prob  # Normalize each probability
            
    sorted_total_probs = sorted(total_probs.items(), key=lambda x: x[1], reverse=True)
    letters = ['A', 'B', 'C', 'D']
    choice_to_index = {letters[i]: i for i in range(len(letters))}
    choice_to_index[None] = len(letters)
    if choice_to_index[data['predict']] == data['label']:
        let, prob = sorted_total_probs[0]
        if prob >= threshold:
            dict_matrix['correct_high'].append(1)
        else:
            dict_matrix['correct_low'].append(1)
    else:
        let, prob = sorted_total_probs[0]
        if prob > threshold:
            dict_matrix['incorrect_high'].append(1)
        else:
            dict_matrix['incorrect_low'].append(1)
    return

In [ ]:
dict_matrix = {}
dict_matrix['correct_high'] = []
dict_matrix['correct_low'] = []
dict_matrix['incorrect_high'] = []
dict_matrix['incorrect_low'] = []

for item in test_data_list:
    get_probs(item, dict_matrix, 0.5)
suma = 0
for k, v in dict_matrix.items():
    suma += sum(v)
for k, v in dict_matrix.items():
    dict_matrix[k] = round(sum(v) / suma, 3)
print(dict_matrix)

In [ ]:
word_play, sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play, sentence_play)
final_result.update(dict_matrix)  # This merges dict_matrix into final_result
save(final_result, name="original", example=get_single_demo(test_data_list[0]), data=test_data_list)